<a href="https://colab.research.google.com/github/sagar-git-23/movie/blob/main/pdcda2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi


Sun Nov  6 12:28:30 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import torch
torch.cuda.is_available()


True

In [3]:
import tensorflow as tf
tf.test.gpu_device_name()


'/device:GPU:0'

In [4]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Sun_Feb_14_21:12:58_PST_2021
Cuda compilation tools, release 11.2, V11.2.152
Build cuda_11.2.r11.2/compiler.29618528_0


In [5]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-6r3lbrs2
  Running command git clone -q https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-6r3lbrs2
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4306 sha256=53c54d2978fecedbc75d715bdfe68b06318186acd5cafb580c8908df637506a8
  Stored in directory: /tmp/pip-ephem-wheel-cache-om0g0__c/wheels/ca/33/8d/3c86eb85e97d2b6169d95c6e8f2c297fdec60db6e84cb56f5e
Successfully built NVCCPlugin


In [6]:
%load_ext nvcc_plugin


created output directory at /content/src
Out bin /content/result.out


In [7]:
%%cu
#include <iostream>
    int main()
{
    std::cout << "Welcome To GeeksforGeeks\n";
    return 0;
}

Welcome To GeeksforGeeks



In [8]:
%%cu
#include<stdio.h>
#include<cuda.h>
__global__ void avg(int *l,int *m, int *n)
{
    int x=blockIdx.x;
    int y=blockIdx.y;
    int id=gridDim.x * y +x;
    n[id]=(l[id]+m[id])/2;
}
int main()
{
    int a[256][256];
    int b[256][256];
    int c[256][256];
    int *d,*e,*f;
    int i,j;
    printf("\n First matrix of size 256 * 256\n");
    for(i=0;i<256;i++)
    {
        for(j=0;j<256;j++)
            {
                a[i][j] = rand()%256;
            }
    }
 for(i=0;i<256;i++)
    {
        for(j=0;j<256;j++)
            {
                printf("%d\t",a[i][j]);
            }
      printf("\n");
    }
    printf("\n Second matrix of size 256 * 256\n");
        for(i=0;i<256;i++)
        {
            for(j=0;j<256;j++)
                {
                 b[i][j] = rand()%256;
                }
        }
 for(i=0;i<256;i++)
    {
        for(j=0;j<256;j++)
            {
                printf("%d\t",b[i][j]);
            }
      printf("\n");
    }
    cudaMalloc((void **)&d,256*256*sizeof(int));
    cudaMalloc((void **)&e,256*256*sizeof(int));
    cudaMalloc((void **)&f,256*256*sizeof(int));
 cudaMemcpy(d,a,256*256*sizeof(int),cudaMemcpyHostToDevice);
 cudaMemcpy(e,b,256*256*sizeof(int),cudaMemcpyHostToDevice);
    
dim3 grid(256,256);
/* Here we are defining two dimensional Grid(collection of blocks) structure. Syntax is dim3 grid(no. of columns,no. of rows) */

    avg<<<grid,1>>>(d,e,f);

 cudaMemcpy(c,f,256*256*sizeof(int),cudaMemcpyDeviceToHost);
    printf("\nAverage of two matrices:\n ");
    for(i=0;i<256;i++)
    {
        for(j=0;j<256;j++)
        {
              printf("%d\t",c[i][j]);
        }
        printf("\n");
    }
    cudaFree(d);
    cudaFree(e);
    cudaFree(f);
    return 0;
}


 First matrix of size 256 * 256
103	198	105	115	81	255	74	236	41	205	186	171	242	251	227	70	124	194	84	248	27	232	231	141	118	90	46	99	51	159	201	154	102	50	13	183	49	88	163	90	37	93	5	23	88	233	94	212	171	178	205	198	155	180	84	17	14	130	116	65	33	61	220	135	112	233	62	161	65	225	252	103	62	1	126	151	234	220	107	150	143	56	92	42	236	176	59	251	50	175	60	84	236	24	219	92	2	26	254	67	251	250	170	58	251	41	209	230	5	60	124	148	117	216	190	97	137	249	92	187	168	153	15	149	177	235	241	179	5	239	247	0	233	161	58	229	202	11	203	208	72	71	100	189	31	35	30	168	28	123	100	197	20	115	90	197	94	75	121	99	59	112	100	36	17	158	9	220	170	212	172	242	27	16	175	59	51	205	227	80	72	71	21	92	187	111	34	25	186	155	125	245	11	225	26	28	127	35	248	41	248	164	27	19	181	202	78	232	152	50	56	224	121	77	61	52	188	95	78	119	250	203	108	5	172	134	33	43	170	26	85	162	190	112	181	115	59	4	92	211	54	148	179	175	226	240	228	158	79	50	21	73	253	130	78	169	
8	112	212	178	138	41	84	72	154	10	188	213	14	24	168	68	172	9